<a href="https://colab.research.google.com/github/restrepo/medicion/blob/master/cienciometria/Query_CTR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Búsquedas WOS+SCI+SCP+PTJ+CTR para UdeA

Búsquedas en bases bibligráficas  
* Web of Science (WOS), 
* Scielo (SCI)
* Scopus  (SCP)
* Puntaje (UDEA)
* Center (CTR)
de los artículos científicos de la UdeA

La base de datos se creó con:

[WOS_SCI_SCP_PTJ_CTR.ipynb](./WOS_SCI_SCP_PTJ_CTR.ipynb)

In [1]:
import os
VERSION='NEW'
if os.getcwd()=='/content':
    !pip install openpyxl xlrd wosplus fuzzywuzzy[speedup] > /dev/null

## functions

In [2]:
import pandas as pd
import wosplus as wp
pd.set_option('display.max_colwidth',200)
from venn import draw_venn, generate_colors
import numpy as np
import fuzzywuzzy.process as fwp
from fuzzywuzzy import fuzz
import re
idc='CÉDULA'

##  Configure public links of  files in Google Drive
* If it is a Google Spreadsheet the corresponding file is downloaded as CSV
* If it is in excel/json or text file the file is downloaded  directly

To define your  own labeled IDs for public google drive files edit the next cell:

In [3]:
%%writefile drive.cfg
[FILES]
WOS_SCI_SCP_PTJ_CTR.json.gz=19E1C1kRk4I0V3uXojqko8-NEicWaPp1j
Base_de_datos_investigadores_Definitiva.csv=12oalgUeKhpvzkTPBP8pXCeHTrF-KO223dy9ov9w9QKs
oaudea.xlsx             = 1CcwobiEFACIbffNzNdLxpdxQukr8cZ5x

Overwriting drive.cfg


##  Load data bases

In [4]:
affil='Univ Antioquia'
drive_files=wp.wosplus('drive.cfg')

In [5]:
UDEAjsonfile='WOS_SCI_SCP_PTJ_CTR.json.gz'
tmp=drive_files.load_biblio(UDEAjsonfile,compression='gzip')
UDEA=drive_files.biblio['WOS'].copy().reset_index(drop=True)

In [6]:
UDEA[UDEA['UDEA_authors']!=''].shape

(12324, 181)

## Normalizations

In [7]:
UDEA['DI']=UDEA.apply( 
     lambda row: row['DI'] if row['DI'] else row['SCP_DOI'],axis=1 ).str.strip().str.lower()
UDEA['DI']=UDEA.apply( 
     lambda row: row['DI'] if row['DI'] else row['SCI_DI'],axis=1 ).str.strip().str.lower()

Normalize 'AU'

In [8]:
UDEA['AU']=UDEA['AU'].apply(lambda l: l if re.search('\n$',l) 
                 else 
                 '\n'.join(
            [
    re.sub( '([\w\-\s]+)(\s[\w]\.)',r'\1,\2',s ,re.UNICODE   ) 
                 for s in l.split(', ')]+['']
           ).replace('.','') )

In [9]:
UDEA[~UDEA['AU'].str.contains('\n$')].shape

(0, 181)

In [10]:
def fix_NOMBRE_COMPLETOS(l):
    for d in l:
        if not d.get('NOMBRE COMPLETO'):
            d['NOMBRE COMPLETO']=''
            if d.get('NOMBRES'):
                d['NOMBRE COMPLETO']=d['NOMBRE COMPLETO']+d.get('NOMBRES')
            if d.get('PRIMER APELLIDO'):
                d['NOMBRE COMPLETO']=d['NOMBRE COMPLETO']+' '+d.get('PRIMER APELLIDO')
            if d.get('SEGUNDO APELLIDO'):
                d['NOMBRE COMPLETO']=d['NOMBRE COMPLETO']+' '+d.get('SEGUNDO APELLIDO')            
    return l

In [11]:
UDEA['UDEA_authors']=UDEA['UDEA_authors'].apply(fix_NOMBRE_COMPLETOS)

In [12]:
dfid=UDEA[['UDEA_nombre','UDEA_cedula']]
dfid=dfid[dfid['UDEA_cedula'].apply(lambda n: n!=0 if isinstance(n,int) else False)].reset_index(drop=True)
dfid=dfid[dfid['UDEA_nombre']!=''].drop_duplicates().reset_index(drop=True)
dfid.shape

(975, 2)

In [13]:
def get_cedula(l,fullnames_with_id=dfid):
    for d in l:
        if not d.get('CÉDULA') and d.get('full_name'):
            try:
                d['CÉDULA']=dfid[ ( dfid['UDEA_nombre']==d.get('full_name')) 
                            & ( dfid['UDEA_cedula']!=0 ) 
                            ].reset_index(
                          drop=True)['UDEA_cedula'].loc[0]
            except KeyError:
                pass
    return l

In [14]:
UDEA['UDEA_authors']=UDEA['UDEA_authors'].apply(get_cedula)

In [15]:
def normalize_NAME(row,WOS_NAME,SCI_NAME,SCP_NAME,UDEA_NAME,
                   SCI_sub=('',''),SCP_sub=('',''),UDEA_sub=('',''),
                  DEBUG=False):
    sn=row[WOS_NAME]
    if not sn:
        if SCI_NAME and row[SCI_NAME]:
            sn=re.sub( SCI_sub[0],SCI_sub[1], str( row[SCI_NAME] ) )
            if DEBUG:
                print('SCI: {}:{}'.format(row[SCI_NAME], sn))
        elif SCP_NAME and row[SCP_NAME]:
            sn=re.sub( SCP_sub[0],SCP_sub[1], str( row[SCP_NAME] ) )
        elif UDEA_NAME and row[UDEA_NAME]:
            sn=re.sub( SCP_sub[0],SCP_sub[1], str( row[UDEA_NAME] ) )
            if DEBUG:
                print('UDEA: {}:{}'.format(row[UDEA_NAME], sn))            
    return sn

In [16]:
WOS_NAME ='SN'
UDEA[WOS_NAME]=UDEA.apply(lambda row: 
                       normalize_NAME(row,WOS_NAME,
                                      SCI_NAME ='SCI_SN',
                                      SCP_NAME ='SCP_ISSN',
                                      UDEA_NAME='UDEA_issn rev',
                                      SCP_sub=( '^([\w]{4})([\w]{4})$',r'\1-\2\n'  ),
                                      UDEA_sub=('(\w)$',r'\1\n')
                                     )
                      ,axis='columns')

In [17]:
WOS_NAME ='DI'
UDEA[WOS_NAME]=UDEA.apply(lambda row: 
                       normalize_NAME(row,WOS_NAME,
                                      SCI_NAME ='SCI_DI',
                                      SCP_NAME ='SCP_DOI',
                                      UDEA_NAME='UDEA_doi')
                      ,axis='columns')

In [18]:
WOS_NAME ='VL'
UDEA[WOS_NAME]=UDEA.apply(lambda row: 
                       normalize_NAME(row,WOS_NAME,
                                      SCI_NAME ='SCI_VL',
                                      SCP_NAME ='SCP_Volume',
                                      UDEA_NAME='',
                                      SCI_sub=('(\w)$',r'\1\n'),
                                      SCP_sub=('(\w)$',r'\1\n')
                                     )
                      ,axis='columns')

In [19]:
WOS_NAME ='IS'
UDEA[WOS_NAME]=UDEA.apply(lambda row: 
                       normalize_NAME(row,WOS_NAME,
                                      SCI_NAME ='SCI_IS',
                                      SCP_NAME ='SCP_Issue',
                                      UDEA_NAME='',
                                      SCP_sub=('(\w)$',r'\1\n')
                                     )
                      ,axis='columns')

In [20]:
WOS_NAME ='BP'
UDEA[WOS_NAME]=UDEA.apply(lambda row: 
                       normalize_NAME(row,WOS_NAME,
                                      SCI_NAME ='SCI_BP',
                                      SCP_NAME ='SCP_Page start',
                                      UDEA_NAME='',
                                      SCP_sub=('(\w)$',r'\1\n')
                                     )
                      ,axis='columns')

In [21]:
WOS_NAME ='EP'
UDEA[WOS_NAME]=UDEA.apply(lambda row: 
                       normalize_NAME(row,WOS_NAME,
                                      SCI_NAME ='SCI_EP',
                                      SCP_NAME ='SCP_Page end',
                                      UDEA_NAME='',
                                      SCP_sub=('(\w)$',r'\1\n')
                                     )
                      ,axis='columns')

In [22]:
UDEA['UDEA_idioma']=UDEA['UDEA_idioma'].apply(lambda s: 'Spanish' if s=='ESPA?' else s)
UDEA['UDEA_idioma']=UDEA['UDEA_idioma'].apply(lambda s: 'English' if s=='INGLE' else s)
WOS_NAME ='LA'
UDEA[WOS_NAME]=UDEA.apply(lambda row: 
                       normalize_NAME(row,WOS_NAME,
                                      SCI_NAME ='SCI_LA',
                                      SCP_NAME ='SCP_Language of Original Document',
                                      UDEA_NAME='UDEA_idioma',
                                      SCI_sub =('\t',r''),
                                      SCP_sub =('(\w)$',r'\1\n'),
                                      UDEA_sub=('(\w)$',r'\1\n')
                                     )
                      ,axis='columns')

## Centros

In [23]:
udea=UDEA.copy()

In [24]:
def convert_dictionary_keys_of_list_of_dictionaries_to_column_dataframe(df,
                            json_column='UDEA_authors',dictionary_key='FACULTAD',sep='; '):
    return df[json_column].apply(lambda l:  
       [d.get(dictionary_key) for d in  l if type(l)==list and d.get(dictionary_key) ]  
               ).apply(
        pd.np.unique
        ).apply(sep.join)    

In [25]:
udea['FACULTAD']=convert_dictionary_keys_of_list_of_dictionaries_to_column_dataframe(udea,
                                        json_column='UDEA_authors',dictionary_key='FACULTAD')

In [26]:
udea['DEPARTAMENTO']=convert_dictionary_keys_of_list_of_dictionaries_to_column_dataframe(udea,
                                        json_column='UDEA_authors',dictionary_key='DEPARTAMENTO')

In [27]:
udea['GRUPO']=convert_dictionary_keys_of_list_of_dictionaries_to_column_dataframe(udea,
                                        json_column='UDEA_authors',dictionary_key='GRUPO')

In [28]:
udea['autores_UDEA']=convert_dictionary_keys_of_list_of_dictionaries_to_column_dataframe(udea,
                                        json_column='UDEA_authors',dictionary_key='full_name')

# Prepare columns

In [29]:
def new_Autores(l):
    dd={}
    ll=[]
    for d in l:
        if d.get('NOMBRE COMPLETO'):
            dd['Nombres']=d['NOMBRE COMPLETO']
            dd['Tipo']='Profesor vinculado'
        if d.get('CÉDULA'):
            dd['Identificación']=d['CÉDULA']
        ll.append(dd)
    return ll

In [30]:
udea['Título del Artículo']=udea['TI']
udea['Nombre de la Revista']=udea['SO']
udea['ISSN']=udea['SN'].str.replace('\n','')
udea['DOI']=udea['DI']
udea['Volumen']=udea['VL'].str.replace('\n','')
udea['Número']=udea['IS'].str.replace('\n','')
udea['Páginas']=udea.apply(lambda row:
                  row['BP'].replace('\n','')+'-'+row['EP'].replace('\n','') 
                  if row['BP'] 
                  else row['BP'],axis='columns').str.replace('\-$','')
udea['Año']=udea['PY']
udea['Semestre']=''
udea['Idioma']=udea['LA'].str.replace('\n','')
udea['Tipo de Revista']=udea.apply(lambda row: 'Revista Internacional Indexada' 
           if row.get('Tipo').find('WOS')>-1 or row.get('Tipo').find('SCP')
           else 'Revista Indexada en SCIELO',axis='columns')
udea['Clasificación Colciencias']=''
udea['Producto asociado a un proyecto de extensión o investigación']=''
udea['Nombre del proyecto de extensión o investigación']=''

In [31]:
udea['Autores']=udea['UDEA_authors'].apply(new_Autores)

In [32]:
udea_au=udea[udea['Autores'].apply(len)>0].reset_index(drop=True)

In [33]:
udea.PY.max()

2018

In [35]:
new_format=pd.read_excel('https://docs.google.com/spreadsheets/d/e/2PACX-1vTiaxuZSGmI-aFgMUVjRAU3ws7WN9xmtjMEWu_SLOd5kAq_ZAuUtJUVr8qxNl3sMcp_fjE2gLVt_tdp/pub?output=xlsx')

In [36]:
udea_au[list(new_format.columns)+[ 'FACULTAD','DEPARTAMENTO','GRUPO' ]].sort_values(
     'FACULTAD',ascending=False).to_excel('UDEA_2018.xlsx',index=False)

In [37]:
new_format[:3]

,Título del Artículo,Nombre de la Revista,ISSN,DOI,Volumen,Número,Páginas,Año,Semestre,Idioma,Tipo de Revista,Clasificación Colciencias,Producto asociado a un proyecto de extensión o investigación,Nombre del proyecto de extensión o investigación,Autores
0,Design and Optimization of a Multi-Element Hydrofoil for a Horizontal-Axis Hydrokinetic Turbine,Energies,1996-1073,10.3390/en12244679,12,24.0,1-18,2019,2019-2,Inglés,Revista Internacional Indexada,A1,No,NaN,"{ Nombres: Jonathan Aguilar Bedoya, Identificación: null, Tipo: Estudiante de posgrado, { undefined: 6, undefined: 0, undefined: 1, undefined: 7, undefined: 8, undefined: -, undefined: M, undefin..."
1,Design of a wave energy converter system for the Colombian Pacific Ocean,Revista Facultad de Ingeniería,0120-6230,10.17533/udea.redin.20190406,94,NaN,8-23,2020,2020-1,Inglés,Revista Internacional Indexada,B,NaN,NaN,"{ Nombres: Edwin lenin Chica Arrieta, Identificación: 78761560, Tipo: Profesor vinculado, { undefined: 5, undefined: 1, undefined: 6, undefined: -, undefined: I, undefined: N, undefined: G, undef..."


In [38]:
new_format.columns

Index(['Título del Artículo', 'Nombre de la Revista', 'ISSN', 'DOI', 'Volumen',
       'Número', 'Páginas', 'Año', 'Semestre', 'Idioma', 'Tipo de Revista',
       'Clasificación Colciencias',
       'Producto asociado a un proyecto de extensión o investigación',
       'Nombre del proyecto de extensión o investigación', 'Autores'],
      dtype='object')

In [39]:
udea_au.shape

(12324, 200)

In [ ]:
def get_cedula(l,fullnames_with_id=dfid):
    for d in l:
        if not d.get('CÉDULA') and d.get('full_name'):
            try:
                d['CÉDULA']=dfid[ ( dfid['UDEA_nombre']==d.get('full_name')) 
                            & ( dfid['UDEA_cedula']!=0 ) 
                            ].reset_index(
                          drop=True)['UDEA_cedula'].loc[0]
            except KeyError:
                pass
    return l